(ws2)=
# Worksheet 2 🐼

:::{epigraph}
Tabular Data and Potential Outcomes

-- TODO your name here
:::

:::{admonition} Collaboration Statement
- TODO brief statement on the nature of your collaboration.
- TODO your collaborator's names here.
:::

# Learning Objectives

- Practice tabular data manipulation through pandas
- Learn more about the potential outcomes framework for causal inference
- Familiarization with mathematical operations on potential outcomes
- Familiarization with Gradescope's autograder

:::{note} Before you begin
Please go to `Edit -> Clear Outputs of All Cells` so you can run the code cells yourself!
:::

# 1. `pandas` and tabular data [2 pts]

To familiarize ourselves with working with tabular data, we will use the [pandas](https://pandas.pydata.org/) library and work with the [National Health and Nutrition Examination Survey (NHANES) dataset](https://www.cdc.gov/nchs/nhanes/about/?CDC_AAref_Val=https://www.cdc.gov/nchs/nhanes/about_nhanes.htm). NHANES is a large, national survey that measures the health and nutrition of adults and children in the United States.

The NHANES dataset is frequently used in **observational studies**, where there is no intervention or manipulation of the data like a randomized experiment. We'll cover these type of studies in a few weeks. For now, we'll use pandas to answer some **descriptive** questions about the data.

## `pandas` basics

Pandas is the defacto Python framework for working with tabular data, and it is supported by a large ecosystem of libraries, including seamless integration with numpy. Pandas provides two main data structures:

- `DataFrame`: a 2-dimensional data structure often used to represent a table with rows and named columns.
- `Series`: a 1-dimensional, **labelled** array, often used to represent a single column or row in a `DataFrame`

Pandas dataframes allow for the datatypes of the columns to be mixed. Let's create a couple of examples by running the code cells below:

In [68]:
import numpy as np

# the standard import idiom for pandas
import pandas as pd

In [ ]:
# Create a series from a list, note how there can 'nan' (missing) values
review_scores = pd.Series([4, np.nan, 2, 3])

# Note that the series can be of different data types
review_text = pd.Series(['I liked it', 'It was awful', 'Bland', 'Pretty good'])

# pandas will automatically assign a data type (dtype) to the series
review_scores

:::{note}
`np.nan` is a special value in pandas that represents missing data. A critical part of the data science workflow in practice is handling missing data, as results can change dramatically when missing data is not handled properly.
:::


We can then create a dataframe by passing in a dictionary of series, where the keys are the column names:

In [ ]:
df = pd.DataFrame({'review_scores': review_scores, 'review_text': review_text})

# Print the dataframe
df

Alternatively, we can create a dataframe from a dictionary of lists, which can sometimes be convenient:

In [71]:
# this code produces the same dataframe as before
df = pd.DataFrame({'review_scores': [4, np.nan, 2, 3], 
                   'review_text': ['I liked it', 'It was awful', 'Bland', 'Pretty good']})


For high-level inspection of the dataframe, we can use the following functions and attributes:

- `df.head()`: returns the first 5 rows of the dataframe
- `df.tail()`: returns the last 5 rows of the dataframe
- `df.info()`: returns a summary of the dataframe, including the number of rows, columns, and the data types of each column
- `df.columns`: returns the column names of the dataframe
- `df.shape`: returns the number of rows and columns in the dataframe
- `df.dtypes`: returns the data types of each column

You can play around with the dataframe in the cell below:

In [ ]:
# the shape is a tuple of (num rows, num columns)
df.shape


Often times we'll be working with pre-existing datasets. Here, we'll use a pre-loaded version of the NHANES dataset, which is available in the `nhanes` package. 

:::{note}
To load data from files manually, pandas provides various `pd.read_*` functions. For example, [pd.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) loads (comma-separated values) CSV files. See pandas' [I/O documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) for more options. However, for the purposes of this worksheet, we'll use a 3rd party package to load in NHANES because it is a complex dataset.
:::


In [ ]:
# load the NHANES data and metadata using pre-built functions
from nhanes.load import load_NHANES_data, load_NHANES_metadata

nhanes_df = load_NHANES_data()

nhanes_df.info()

From above, we see that the dataframe has 8,366 rows and 197 columns. Here, each row represents a survey respondent, and each column represents a survey question that was asked.

Taking a look at the columns, we see that there are a wide range of health-related questions that were asked:

In [ ]:
# .values returns the column names as a numpy array
# nhanes_df.columns.values

## Data cleaning

To select a single column, we can square bracket indexing, as if we were acessing a dictionary:

In [ ]:
# Selects the 'GeneralHealthCondition' column and prints the first 5 rows
nhanes_df['GeneralHealthCondition'].head()

When initially exploring a dataset, it is often useful to see the unique values in a column:

In [ ]:
# Get the unique values in the 'GeneralHealthCondition'
nhanes_df['GeneralHealthCondition'].unique()


Notice two things about the `GeneralHealthCondition` column:

- There are some missing values in the column, represented as `nan` 
- There are two values that seem to be typos: `Poor?` and `Fair or`


To simplify our analysis, we can remove the rows with missing values via [dropna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html):

In [77]:
# remove rows with missing values in the 'GeneralHealthCondition' column, resulting in a new dataframe
nhanes_df = nhanes_df.dropna(subset=['GeneralHealthCondition'])

We can also correct the typos using [replace()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html):

In [ ]:
# replace the typos with the correct values
nhanes_df['GeneralHealthCondition'] = nhanes_df['GeneralHealthCondition'].replace({'Poor?': 'Poor', 'Fair or': 'Fair'})

# print the unique values again to see the changes
nhanes_df['GeneralHealthCondition'].unique()

:::{tip}
Notice how in both operations, we reassign the result back to `nhanes_df` or a column in `nhanes_df`, which is a common pattern in pandas -- operations often return a new dataframe or column, and we need to reassign the result back to the original variable if we want to keep the changes.
:::


## Indexing and assignment

The square bracket indexing can be generalized to selecting multiple columns:

In [ ]:
# Selects multiple columns and prints the first 10 rows
cols = ['GeneralHealthCondition', 'AgeInYearsAtScreening', 'SmokedAtLeast100CigarettesInLife', 'WeightKg', 'VigorousRecreationalActivities']
nhanes_df[cols].head(10)

Just like numpy, we can also use boolean indexing to select portions of the dataframe based on a condition:

In [ ]:
# Selects respondents who have smoked at least 100 cigarettes in their life
sel_df = nhanes_df[nhanes_df['SmokedAtLeast100CigarettesInLife'] == 1]

sel_df[cols].head(10)

We can then use the `value_counts` function to get the frequency of each category in a column:

In [ ]:
print(sel_df['GeneralHealthCondition'].value_counts())

# normalize=True to get the proportion of each category
# dropna=False to include missing values as part of counts
print(sel_df['GeneralHealthCondition'].value_counts(dropna=False, normalize=True))

:::{tip}
Many pandas operations by default drop na values, so be careful to double check what the default behavior is!
:::

**1.1**. How would we get the general health condition proportions of respondents who have engaged in vigorous recreational activities (1 = yes, 0 = no)? Write code to do this below:

In [ ]:
# TODO your code here

What proportion of respondents who have engaged in vigorous recreational activities report being in excellent health?

**Your response**: TODO

These boolean conditions can be combined using the `&` (AND), `|` (OR), and `~` (NOT) operators. Additionally, there are some special functions that can be used to select data based on a condition:

In [83]:
# Combining conditions: respondents who are 18 or older and have engaged in vigorous recreational activities
nhanes_df[(nhanes_df['AgeInYearsAtScreening'] >= 18) & (nhanes_df['VigorousRecreationalActivities'] == 1)] 

# isin(): select rows where a column is in a list of values
nhanes_df[nhanes_df['GeneralHealthCondition'].isin(['Poor?', 'Fair or'])]

# isna(): select rows where a column is missing
nhanes_df[nhanes_df['SmokedAtLeast100CigarettesInLife'].isna()];

:::{tip}
To avoid errors, always use parentheses when combining conditions:

- Incorrect: `df[column1 == value1 & column2 == value2]`
- Correct: `df[(column1 == value1) & (column2 == value2)]`
:::


We can also create new columns or modify existing columns via `=` assignment (like we did with `replace()`):

In [ ]:
# Assign a constant value to a new column where every row has the same value
nhanes_df['country'] = 'United States'

# Create a new column based on a condition of other columns
nhanes_df['is_adult'] = nhanes_df['AgeInYearsAtScreening'] >= 18

# Create a new column based on a function of other columns: calculate BMI
nhanes_df['bmi'] = nhanes_df['WeightKg'] / (nhanes_df['StandingHeightCm'] / 100) ** 2

# Print the updated dataframe
nhanes_df[['country', 'is_adult', 'AgeInYearsAtScreening', 'bmi']].head()

**1.2**. Let's practice combining these operations. Select respondents who:

- are adults AND
- report being in `Excellent` or `Very good` health AND
- are NOT missing any values in the `bmi` column 

Write code to do this below:

In [ ]:
# TODO your code here

How many respondents are in the selected group? How many of them have smoked at least 100 cigarettes in their life?

**Your response**: TODO


## the `.loc` operator

In addition to square bracket indexing, pandas provides a powerful indexing method via the [.loc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html) operator, which allows us to select rows and columns by their labels or even boolean conditions.

Generally, the `.loc` operator is used to select rows and columns by their labels:

```python
df.loc[row_label, column_label]
```

The loc operator borrows from the slice notation of python lists, where we can use `:` to select all rows or column:

In [ ]:
# selects all rows and the 'bmi' column, returned as a series
nhanes_df.loc[:, 'bmi'].head()


In [ ]:
# selects all rows and the ['country', 'is_adult', 'bmi'] columns
# Note that the columns are wrapped in a list!
nhanes_df.loc[:, ['country', 'is_adult', 'bmi']].head()

Where the .loc operator becomes useful is when we want to compactly select rows and columns based on a condition:

In [ ]:
# selects rows where the 'bmi' column is greater than 25, and returns the 'bmi' and 'is_adult' columns
nhanes_df.loc[nhanes_df['bmi'] > 25, ['bmi', 'is_adult']].head()

This selection can be used to modify columns based on a condition:

In [ ]:
# set the 'bmi' column to np.nan for all rows where the 'is_adult' column is False
nhanes_df.loc[nhanes_df['is_adult'] == False, 'bmi'] = np.nan

# print the updated dataframe
nhanes_df[['bmi', 'is_adult']].head()

**1.3.** Common age cutoffs that are used in health policy studies are 18 and 65 -- 18 is when people are legally considered adults in the United States, and 65 is when they are eligible for Medicare health insurance. Create a new column `age_category` using the `AgeInYearsAtScreening` column and the `.loc` operator that makes the following assignments:

- `'child'`: age < 18
- `'adult'`: 18 <= age < 65
- `'senior'`: age >= 65

In [ ]:
# TODO your code here

How many respondents are in the 'senior' age category?

**Your response**: TODO

## Grouping and transformation
 
The [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function is a powerful tool for performing aggregations on subsets of the dataframe. We pass in one or more columns as the `by` argument, which then divides the original Dataframe based on the unique values of the column(s). We then often apply an **aggregation** function to each group, resulting in a new dataframe.

We can replicate `value_counts()` with `groupby()` and the `size()` aggregation function:

In [ ]:
# Count number of people in each health condition category
print(nhanes_df.groupby(by='GeneralHealthCondition').size())


We can also compute summary statistics like `mean()`, `std()`, `median()`, `min()`, `max()` on each group:

In [ ]:
# Calculate mean BMI by general health condition
nhanes_df.groupby(by='GeneralHealthCondition')['bmi'].mean()

We can also group by multiple columns, which then groups the dataframe by the unique cross-product of the columns:

In [ ]:
# Calculate mean BMI by general health condition and vigorous recreational activities
# as_index=False keeps the groupby columns as columns in the resulting dataframe
nhanes_df.groupby(by=['GeneralHealthCondition', 'VigorousRecreationalActivities'], as_index=False)['bmi'].mean()

To apply multiple aggregation functions, we can pass in a dictionary of columns as keys and functions as values to [agg()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html):

In [ ]:
# Calculate mean, min, max for BMI and median household income by general health condition
nhanes_df.groupby(by='GeneralHealthCondition').agg({'bmi': ['mean', 'min', 'max'], 'AnnualHouseholdIncome': 'median'})

**1.4**. Generate a table that groups by:

- `GeneralHealthCondition`
- `SmokedAtLeast100CigarettesInLife`

and calculates the mean of the following columns:

- `EverToldYouHavePrediabetes`: 0 = no, 1 = yes
- `EverToldYouHadHeartAttack`: 0 = no, 1 = yes

:::{note}
The mean of a binary column is equivalent to the proportion of 1s in the column.
:::

In [ ]:
# TODO your code here


Which subgroup of health conditions and smoking status has the highest proportion of people who have been told they have had a heart attack?

**Your response**: TODO


:::{note}
For more on pandas operations, see [this quickstart guide](https://pandas.pydata.org/docs/user_guide/10min.html) and the associated links within it.
:::


# 2. Reading: Neal 2020 [1 pt]

Read the following sections of [Neal 2020: An Introduction to Causal Inference](https://www.bradyneal.com/Introduction_to_Causal_Inference-Dec17_2020-Neal.pdf) and answer the questions below:

- Sections 2.1 - 2.3.2
- Section 2.3.5

:::{admonition} Reading Notes

- Don't worry too much about the causal graphs shown in the margin, as we'll cover causal graph formalisms in upcoming classes.

- The notation $\triangleq$ is used to denote "is defined as". For example, Equation 2.1 $\tau_i \triangleq Y_i(1) - Y_i(0)$ means that the individual treatment effect for unit $i$ is defined as $Y_i(1) - Y_i(0)$.

- A mathematical identity we'll need for the derivations is **linearity of expectation**. This states that for *any* random variables $X$ and $Y$ and constant $a$: 

$$
E[X + Y] = E[X] + E[Y]
$$

$$
E[aX] = aE[X]
$$

- Implications of two variables $X$ and $Y$ being independent (written $X \perp Y$) are that:

$$
E[X | Y=y] = E[X]
$$

$$
E[XY] = E[X]E[Y]
$$

- The consistency assumption (Assumption 2.5) allows us to define the observed outcome $Y$ with binary treatment $T$ as follows:

$$
Y = T \cdot Y(1) + (1-T) \cdot Y(0)
$$
:::



**2.1.** State the fundamental problem of causal inference in your own words.

**Your Response**: TODO

**2.2.** These sections of Neal 2020 cover three key assumptions that allow us to **identify** (definition 2.1, p. 10) the average treatment effect (ATE) of a binary treatment:

- ignorability / exchangeability (Assumption 2.1, p. 9)
- no interference (Assumption 2.4, p. 13)
- consistency (Assumption 2.5, p. 14)

Below is a restatement of the ATE derivation. State which mathematical identities and assumptions are used on each line:

$$
ATE &= E[Y(1) - Y(0)]\\
    &= E[Y(1)] − E[Y(0)], \text{ from TODO} \\
    &= E[Y(1) | T = 1] − E[Y(0) | T = 0], \text{ from TODO} \\
    &= E[Y | T = 1] − E[Y | T = 0] \text{ from TODO}
$$

**2.3.** The no interference assumption states that the potential outcomes for one unit are unaffected by the treatment assignment of other units. Breaking down Neal's example on p. 13, suppose that you are unit 1 and your friend is unit 2. If your friend gets a dog ($t_2 = 1$), we can then examine your potential outcomes:

$$
&Y_1(t_1, t_2), \text{ potential outcomes for unit 1} \\ 
&Y_1(0, 1), \text{ you don't get a dog, but your friend does} \\
&Y_1(1, 1), \text{ you and your friend get a dog}
$$

The argument is that it is likely that the potential outcomes for your happiness are affected by your friend's choice to get a dog: perhaps you'll already be happier if your friend gets a dog because you get to play with them, or perhaps you're allergic to dogs so you have to see your friend less often, etc. This leads to a violation of the no interference assumption where your potential outcomes are no longer just a function of your own treatment assignment $t_1$:

$$
Y_1(0, 1) \neq Y_1(0) \\
Y_1(1, 1) \neq Y_1(1)
$$

Neal then states that "violations of the no interference assumption are rampant in network data," e.g. friend groups like the dog example above, social networks, or transportation networks (think roads, subway lines, airports). Give an example of a setting along with a treatment $T$ and outcome $Y$ where you think the no interference assumption is likely to be violated.

**Your Response**: 

- Treatment: TODO
- Outcome: TODO
- Reasoning: TODO

# 3. Causal inference as a missing data problem [1 pt]

Let's now put the conceptual ideas around potential outcomes together with our table manipulation practice in pandas.

Our framework for causal inference is that there is there is a causal "ground truth" table of potential outcomes for each unit. With our example from class, we might be interested in studying the effect of taking aspirin $T$ ($T=1$ means taking aspirin, $T=0$ means not taking aspirin) on headaches $Y$ ($Y=1$ means having a headache, $Y=0$ means not having a headache). The potential outcomes then correspond to:

$$
Y(0) = \text{what the outcome } \textit{would have been} \text{ if } T = 0 \\
Y(1) = \text{what the outcome } \textit{would have been} \text{ if } T = 1
$$

We then have a table of potential outcomes for each unit, where each row is a unit and each column is a potential outcome:

$$
\begin{array}{c|c|c|c}
\text{Unit } i & T & Y(0) & Y(1) \\
\hline
1 & 0 & 1 & 1 \\
2 & 1 & 1 & 0 \\
3 & 0 & 1 & 0 \\
4 & 1 & 0 & 0 \\
\end{array}
$$

The problem is that we are not omniscient, nor do we have a time machine, so we only observe one realized outcome $Y$ for each unit. There are thus missing values in the table of the data we actually observe:

$$
\begin{array}{c|c|c|c|c}
\text{Unit } i & T & Y(0) & Y(1) & Y \\
\hline
1 & 0 & 1 & \text{nan} & 1 \\
2 & 1 & \text{nan} & 0 & 0 \\
3 & 0 & 1 & \text{nan} & 1 \\
4 & 1 & \text{nan} & 0 & 0 \\
\end{array}
$$

The challenge in causal inference is to try to construct causal estimates when we only have the observed data. Throughout this course we'll use our ability to simulate different data generating processes in code to understand the properties of causal inference methods, and when they can or cannot identify the causal effect.

We'll start by writing a utility function that takes treatment assignments and a ground truth table of potential outcomes, and generates the realized outcomes.

:::{admonition} Hint
:class: tip

To construct the observed outcome $Y$, look to how we defined $Y$ under the consistency assumption.

:::

In [96]:
def generate_observed_df(po_df, treatments):
    """
    Generates a dataframe of realized outcomes from a ground truth table of potential outcomes and treatment assignments.
    The unobserved potential outcomes are set to np.nan.
    
    Args:
        po_df (pd.DataFrame): a dataframe of potential outcomes with columns 'Y0' and 'Y1'
        treatments (pd.Series): a series of treatment assignments with the same length as po_df

    Returns:
        pd.DataFrame: a dataframe of realized outcomes with columns 'T', 'Y0', 'Y1', and 'Y'
    """
    assert po_df.shape[0] == treatments.shape[0], "The number of rows in po_df and treatments must match"
    
    # TODO your code here
    observed_df = po_df.copy()

    return observed_df

# Generate some simple test data, matching the example above
treatments = pd.Series([0, 1, 0, 1])
po_df = pd.DataFrame({'Y0': [1, 1, 1, 0], 'Y1': [1, 0, 0, 0]})
observed_df = generate_observed_df(po_df, treatments)
assert 'Y' in observed_df.columns, "The observed dataframe does not have a 'Y' column"

:::{tip}
This is the first auto-graded function in the course -- there will be no submission limit so feel free to try multiple times. We will frequently provide some simple test cases via `assert` statements to check your function. The syntax for assert statements is as follows:

```python
assert bool_expression, description
```
where the `bool_expression` is a boolean expression that tests some condition, and the `description` is a string that will be displayed if the assertion fails (evaluates to `False`).

You are encouraged to write your own test cases as well for your functions!
:::

:::{admonition} Takeaway

This transformation from the ground truth table of potential outcomes to the observed data is a key step in thinking about causal quantities. **If only** we had the ground truth table, we could directly estimate quantities like $E[Y(1)]$ and $E[Y(0)]$. However, since we only have the observed data, we need to make assumptions, like the ones covered in the Neal reading, in order to "reconstruct" those quantities. This is the process of **identifying** the causal effect. See the figure below:

```{figure} ../images/ws2_hernan_fig_1_1.png
---
width: 500px
name: potential-outcomes-to-observed
---
Adapted from Hernán and Robins 2020 1.1
```

We saw from the Neal reading that the assumptions made in a randomized experiment allow for us to identify the causal effect, i.e. reconstruct $E[Y(1)]$ from $E[Y|T=1]$, and $E[Y(0)]$ from $E[Y|T=0]$. 
You may sometimes hear that "association is causation, in a randomized experiment." 

The crux of the "design" step in the causal roadmap generally involves evaluating what assumptions we need in order to perform identification, and whether those assumptions are reasonable to make.
:::

# 4. Reflection [1 pt]

**4.1** How much time did it take you to complete this worksheet? How long did it take you to read the selected sections of Neal 2020?

**Your Response**: TODO

**4.2** What is one thing you have a better understanding of after completing this worksheet and going though the class content this week? This could be about the concepts, the reading, or the code.

**Your Response**: TODO

**4.3** What questions or points of confusion do you have about the material covered in the past week of class?

**Your Response**: TODO


:::{tip}
Don't forget to check that all of the TODOs are filled in before you submit!
:::

# Acknowledgements

Portions of this worksheet are adapted from Bhargavi's study notes on pandas. This worksheet also uses Russ Poldrack's [nhanes package](https://github.com/poldrack/nhanes), which processes the NHANES dataset into a convenient pandas dataframe. 